In [1]:
BASE_DIR = "/Users/denis/MyProjects/Summarization"
GRAPH_BASE_DIR = "/Users/denis/MyProjects/uncased_L-12_H-768_A-12"

In [2]:
import os
from bert_experimental.feature_extraction.bert_feature_extractor import BERTFeatureExtractor
from tree_rank_summaraiser import TreeRankSummariser
import global_const
import pandas as pd
import glob
import time
import logging
import numpy as np
logging.basicConfig(level=logging.DEBUG)

In [3]:
GRAPH_PATH = os.path.join(GRAPH_BASE_DIR, "model_for_inference.pbtxt")
VOCAB_PATH = os.path.join(GRAPH_BASE_DIR, "vocab.txt")

encoder = BERTFeatureExtractor(GRAPH_PATH, VOCAB_PATH)

2020-05-10 14:08:27,558 :  From /Users/denis/anaconda3/envs/Summarization/lib/python3.6/site-packages/bert_experimental/feature_extraction/text_preprocessing.py:287: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


2020-05-10 14:08:27,670 :  From /Users/denis/anaconda3/envs/Summarization/lib/python3.6/site-packages/bert_experimental/feature_extraction/bert_feature_extractor.py:52: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.


2020-05-10 14:08:27,672 :  From /Users/denis/anaconda3/envs/Summarization/lib/python3.6/site-packages/bert_experimental/feature_extraction/bert_feature_extractor.py:38: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.


2020-05-10 14:08:27,689 :  Using temporary folder as model directory: /var/folders/lx/02q9j0f52ds3f_h7jnq_dvmm0000gp/T/tmposfesh7k
2020-05-10 14:08:27,694 :  Using config: {'_model_dir': '/var/folders/lx/02q9j0f52ds3f_h7jnq_dvmm0000gp/T/tmposfesh7k', '_

In [4]:
summariser = TreeRankSummariser(3, encoder)

In [19]:
pattern_str = "/Users/denis/MyProjects/Summarization/data/Companies' news/Norwegian Cruise Line/*"
files_list = glob.glob(pattern_str)
data_list = []
company_name_list = ["Norwegian Cruise Line", "Norwegian Cruise"]
# company_name_list = ["Carnival"]
# company_name_list = ["Pfizer"]

companies = [
    {
        global_const.COMPANY_ISIN: "Norwegian Cruise Line",
        global_const.COMPANY_NAMES: ["Norwegian Cruise Line", "Norwegian Cruise"]
     },
    {
        global_const.COMPANY_ISIN: "Carnival corporation",
        global_const.COMPANY_NAMES: ["carnival corporation", "ccl"]
    }
]

time_list = []
for path in files_list:
    print(path)
    with open(path) as file:
        article_str = file.read()
    
    start = time.time()
    article_sum_dict = summariser.get_summary_for_companies(article_str, companies)
    finish = time.time()
    time_list.append(finish - start)
    for isin, summary in article_sum_dict.items():
        data_list.append({
            "News": article_str,
            "Summarization": summary,
            "isin": isin
        })
    
pd.DataFrame(data=data_list).to_excel(f"/Users/denis/MyProjects/Summarization/data/Companies' news/Norwegian Cruise Line.xls")

DEBUG:tree_rank_summaraiser:Extracted 16 sentences
DEBUG:tree_rank_summaraiser:Pattern: (^|[\.,":'!?()\s])(norwegian cruise line|norwegian cruise)($|[\.,":'!?()\s])
DEBUG:tree_rank_summaraiser:Sentence 0 got weight. ISIN - Norwegian Cruise Line
DEBUG:tree_rank_summaraiser:nclh investor alert: bronstein, gewirtz & grossman, llc reminds norwegian cruise line holdings ltd. investors of class action and encourages investors with losses in excess of $100,000 to contact the firm.
DEBUG:tree_rank_summaraiser:Sentence 1 got weight. ISIN - Norwegian Cruise Line
DEBUG:tree_rank_summaraiser:new york, april 29, 2020 (globe newswire) -- attorney advertising -- bronstein, gewirtz & grossman, llc reminds investors that a class action lawsuit has been filed against norwegian cruise line holdings ltd. ("norwegian" or the company") (nyse:nclh) and certain of its officers, on behalf of shareholders who purchased or otherwise acquired norwegian securities between february 20, 2020 and march 12, 2020, both

/Users/denis/MyProjects/Summarization/data/Companies' news/Norwegian Cruise Line/NCLH INVESTOR ALERT: Bronstein, Gewirtz & Grossman


In [12]:
time_np = np.array(time_list)

In [13]:
time_np.mean()

1.1565142556240684

In [14]:
np.median(time_np)

1.1997120380401611

In [15]:
time_np

array([1.01074505, 0.93726516, 0.58451486, 0.45749283, 2.37396479,
       1.2654283 , 1.59219909, 0.95171022, 1.19971204, 1.28116202,
       1.62971997, 0.78404903, 1.44331813, 1.29001617, 1.02800989,
       1.32319903, 0.87023902, 0.59609008, 1.35493517])